In [1]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential
from keras.applications import InceptionV3, VGG19
from keras.layers import TimeDistributed

import numpy as np

Using TensorFlow backend.


<img src="https://github.com/imatge-upc/activitynet-2016-cvprw/blob/master/misc/images/network_pipeline.jpg?raw=true" height="424" width="400">

In [2]:
# # define CNN model
#cnn = Sequential()
#cnn.add(Conv2D(...))
#cnn.add(MaxPooling2D(...))
#cnn.add(Flatten())
# #define LSTM model
#model = Sequential()
#model.add(TimeDistributed(cnn, ...))
#model.add(LSTM(..))
#model.add(Dense(...))

In [3]:
#model = Sequential()
# # define CNN model
#model.add(TimeDistributed(Conv2D(...))
#model.add(TimeDistributed(MaxPooling2D(...)))
#model.add(TimeDistributed(Flatten()))
# # define LSTM model
#model.add(LSTM(...))
#model.add(Dense(...))

In [4]:
K.set_learning_phase(1)
## Define vision model
## Inception (currently doesn't work)
cnn = InceptionV3(weights='imagenet',
                  include_top='False',
                  pooling='avg')

# Works
#cnn = VGG19(weights='imagenet',
#            include_top='False', pooling='avg')

cnn.trainable = False

H=W=229
C = 3
video_input = Input(shape=(None,H,W,C), name='video_input')

encoded_frame_sequence = TimeDistributed(cnn)(video_input)

encoded_video = LSTM(256)(encoded_frame_sequence)

output = Dense(256, activation='relu')(encoded_video)

video_model = Model(inputs=[video_input], outputs=output)

print(video_model.summary())

video_model.compile(optimizer='adam', loss='mean_squared_error')

#features = np.empty((0,1000))

n_samples = 3
n_frames = 50

frame_sequence = np.random.randint(0.0,255.0,size=(n_samples, n_frames, H,W,C))

y = np.random.random(size=(3,256,))
y = np.reshape(y,(-1,256))

print(frame_sequence.shape)

video_model.fit(frame_sequence, y, validation_split=0.0,shuffle=False, batch_size=1, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
video_input (InputLayer)     (None, None, 229, 229, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1000)        23851784  
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               1287168   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
Total params: 25,204,744
Trainable params: 1,352,960
Non-trainable params: 23,851,784
_________________________________________________________________
None
(3, 50, 229, 229, 3)
Epoch 1/10
3/3 [==============================] - 65s 22s/step - loss: 0.3302
Epoch 2/10
3/3 [==============

In [5]:
x = np.random.randint(0.0,255.0,size=(1, n_frames, H,W,C))
result = video_model.predict(x, batch_size=32, verbose=0)
print(result)

[[ 0.48865858  0.          0.5294463   0.          0.          0.32520968
   0.41552249  0.38604072  0.59496617  0.          0.          0.
   0.53711337  0.          0.41205114  0.56040323  0.54801589  0.
   0.60234529  0.24092622  0.46270445  0.71826959  0.72436512  0.
   0.50306886  0.10389024  0.55030888  0.68757224  0.62232822  0.77048045
   0.22982058  0.28708664  0.4941746   0.68119562  0.          0.
   0.57382393  0.26470238  0.51085168  0.57487714  0.64305335  0.30172488
   0.          0.          0.32503846  0.53593308  0.          0.64809525
   0.56800669  0.09362738  0.35162503  0.28015548  0.70102513  0.31345251
   0.30695018  0.49150968  0.          0.28290972  0.51433855  0.62096369
   0.          0.57412344  0.20297305  0.          0.58501166  0.48780191
   0.30830514  0.73867971  0.32771659  0.28539708  0.5090133   0.34430093
   0.48529592  0.          0.          0.19420263  0.45344388  0.3176997
   0.4220477   0.36404058  0.          0.32214198  0.          0.392251